In [1]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
def clean(text):
    """ Function to clean the text """
    text = text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    texter = re.sub(r"<br />", " ", text)
    texter = re.sub(r"&quot;", "\"",texter)
    texter = re.sub('&#39;', "\"", texter)
    texter = re.sub('\n', " ", texter)
    texter = re.sub(' u '," you ", texter)
    texter = re.sub('`',"", texter)
    texter = re.sub(' +', ' ', texter)
    texter = re.sub(r"(!)\1+", r"!", texter)
    texter = re.sub(r"(\?)\1+", r"?", texter)
    texter = re.sub('&amp;', 'and', texter)
    texter = re.sub('\r', ' ',texter)
    
    # Remove numbers from string
    texter = re.sub(pattern=r"[+-]?\d+(?:\.\d+)?", repl="", string=texter, count=0, flags=0)
    texter = texter.replace("  ", " ")
    clean = re.compile('<.*?>')
    texter = texter.encode('ascii', 'ignore').decode('ascii')
    texter = re.sub(clean, '', texter)
    texter = re.sub(r'[^\w\s]', '', texter)
    if texter == "":
        texter = ""
    return texter

In [4]:
glove_dir = './embeddings/glove.6B.300d.txt'
input_dim = 300

vocab = {}
with open(glove_dir, encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        vocab[word] = vector
f.close()

print('Found %s word vectors.' %len(vocab))

Found 400000 word vectors.


In [54]:
max_len = 50
zero_padding = [0]*input_dim

def get_embeddings(text, emb="LSTM"):
    if emb == "DNN":
        embedding = [0]*input_dim
        i = 0
        for word in text.split(' '):
            if word in vocab:
                i += 1
                embedding += vocab[word]

        if i != 0:
            embedding /= i

        scale = 0.3 # noise weight_scale
        noise = np.random.randn(input_dim)*scale
        embedding += noise
    elif emb == "LSTM":
        embedding = []
        i = 0
        for word in text.split(' '):
            if i == max_len:
                break
            if word in vocab:
                i += 1
                embedding.append(vocab[word])
        
        while i < max_len:
            i += 1
            embedding.append(zero_padding)
            
    return embedding

In [55]:
def transform(X, emb="LSTM"):
    embeddings = []
    for item in X:
        item = clean(item)
        embedding = get_embeddings(item, emb)
        embeddings.append(embedding)
    
    return embeddings

In [56]:
## Create embeddings for input
cleaned_X = np.array(transform(df['text'], "DNN"))
# cleaned_X = np.array(transform(df['text']))
y = np.array(df['target'])

In [57]:
## Sentence BERT

# sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
# sentence_embeddings = sbert_model.encode(cleaned_x, show_progress_bar=True)

In [58]:
# Hyperparameters
num_epochs = 15
batch_size = 128
learning_rate = 0.00001
dropout = 0.25

In [59]:
X_train, X_test, y_train, y_test = train_test_split(cleaned_X, y, test_size=0.15, random_state=42)

X_tr = torch.tensor(X_train, dtype=torch.float)
y_tr = torch.tensor(y_train)
train = TensorDataset(X_tr, y_tr)
trainloader = DataLoader(train, batch_size=batch_size)

X_te = torch.tensor(X_test, dtype=torch.float)
y_te = torch.tensor(y_test)
test = TensorDataset(X_te, y_te)
testloader = DataLoader(test)

In [60]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(300, 500)
        self.hidden1 = nn.Linear(500, 500)
        self.hidden2 = nn.Linear(500, 500)
        self.fc2 = nn.Linear(500, 2)
        self.dropout = nn.Dropout(0.25)
        
        self.batchnorm1 = nn.BatchNorm1d(500)
        self.batchnorm2 = nn.BatchNorm1d(500)
        self.batchnorm3 = nn.BatchNorm1d(500)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = F.relu(self.hidden1(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = F.relu(self.hidden2(x))
        x = self.batchnorm3(x)
        x = self.dropout(x)
#         x = self.fc2(x)
        x = F.log_softmax(self.fc2(x), dim=1)
        return x

net = Net()

In [61]:
# Loss function
criterion = nn.CrossEntropyLoss()

# create your optimizer
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [62]:
# Training
net.train()
training_loss = []
for epoch in range(num_epochs):
    net.train()
    for i, data in enumerate(trainloader):
        inputs, labels = data

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        training_loss.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, loss.data))
        
    # Validation
    net.eval()
    outputs = net(X_te)
    _, predicted = torch.max(outputs, 1)
    total = y_te.size(0)
    correct = (predicted == y_te).sum()

    print(f'Accuracy of the model is: {100*correct/total:.2f}%')


Epoch [1/15], Loss: 0.7227
Accuracy of the model is: 55.52%
Epoch [2/15], Loss: 0.7320
Accuracy of the model is: 59.81%
Epoch [3/15], Loss: 0.7859
Accuracy of the model is: 61.38%
Epoch [4/15], Loss: 0.8178
Accuracy of the model is: 64.01%
Epoch [5/15], Loss: 0.7211
Accuracy of the model is: 65.59%
Epoch [6/15], Loss: 0.8369
Accuracy of the model is: 66.02%
Epoch [7/15], Loss: 0.7481
Accuracy of the model is: 67.43%
Epoch [8/15], Loss: 0.7344
Accuracy of the model is: 67.95%
Epoch [9/15], Loss: 0.8260
Accuracy of the model is: 68.21%
Epoch [10/15], Loss: 0.7284
Accuracy of the model is: 68.65%
Epoch [11/15], Loss: 0.7649
Accuracy of the model is: 68.65%
Epoch [12/15], Loss: 0.6236
Accuracy of the model is: 68.65%
Epoch [13/15], Loss: 0.6260
Accuracy of the model is: 69.53%
Epoch [14/15], Loss: 0.7033
Accuracy of the model is: 69.26%
Epoch [15/15], Loss: 0.6308
Accuracy of the model is: 69.61%


In [16]:
f1_score(predicted, y_te)

0.720510095642933

In [194]:
df_test = pd.read_csv('test.csv')
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [195]:
cleaned_X_test = np.array(transform(df_test['text'], "DNN"))
X_test = torch.tensor(cleaned_X_test, dtype=torch.float)

In [196]:
# Testing
net.eval()
outputs_test = net(X_test)

_, predicted_test = torch.max(outputs_test, 1)

In [197]:
predicted_test.shape

torch.Size([3263])

In [198]:
data = {'id': np.array(df_test['id']),
       'target': np.array(predicted_test)}

In [199]:
df_submission = pd.DataFrame(data)
df_submission.to_csv('submission_lstm.csv', encoding='utf-8', index=False)